# QMMM workflow using LAMMPS and VOTCA-XTP

## What is this tutorial about
In this tutorial, we will learn how to set and perform excited state calculation using the Votca XTP library. We will use thiophene as our QM region.

## Requirements
* You will need to install **VOTCA** using the instructions described [here](https://github.com/votca/votca/blob/master/share/doc/INSTALL.rst)
* Once the installation is completed you need to activate the VOTCA enviroment by running the `VOTCARC.bash` script that has been installed at the bin subfolder for the path that you have provided for the installation step above

## Interacting with the XTP command line interface
The XTP package offers the following command line interface that the user can interact with:
* [xtp_map](https://www.votca.org/xtp/xtp_map.html)
* [xtp_parallel](https://www.votca.org/xtp/xtp_parallel.html)
* [xtp_run](https://www.votca.org/xtp/xtp_run.html)
* [xtp_tools](https://www.votca.org/xtp/xtp_tools.html)

Run the following command to view the help message of `xtp_tools`:

In [ ]:
!xtp_tools -h

### Note
> * In Jupyter the `!` symbol means: *run the following command as a standard unix command*
> * In Jupyter the command `%env` set an environmental variable

## Setting the environment


Remove previous hdf5 file

In [ ]:
!rm -f state.hdf5

## Generate the topology from the Gromacs file
We will first generate the mapping from MD coordinates to segments, creating an [hdf5 file](https://www.hdfgroup.org/solutions/hdf5/) to store the results. You can explore the generated `state.hdf5` file with e.g. hdf5itebrowser. In Python, you can use the [h5py library](https://www.h5py.org/). The command to generate the mapping is the following,

In [ ]:
!xtp_map -v -t MD_FILES/newfile.data -c MD_FILES/traj.dump -s system.xml -f state.hdf5 -i 99 > mapping.out

##  Check the mapping

Let us first output `.pdb` files for the segments, qmmolecules and classical segments in order to check the mapping. So we have to pass the calculator the filename. Votca has two ways to specify options for calculators. Using a file with the `-o` option or for quick things using the `-c` option on the command line, we will use both.

In the [mapchecker section of the manual](https://www.votca.org/xtp/mapchecker.html) you can find a table with the `mapchecker` input variables and their corresponding defaults. Finally, the following command run the check

In [ ]:
!xtp_run -e mapchecker -c map_file=system.xml -f state.hdf5

## Neighborlist Calculation
The following step is to determine the neighbouring pairs for exciton transport.

We will use a cutoff of 1.5 nm. If you want to have a look at an option just the `-d` option with the calculator name

In [ ]:
!xtp_run -d neighborlist

In [ ]:
!xtp_run -e neighborlist -c constant=1.5 -f state.hdf5

## Read reorganization energies
In this step we will read the in site reorganization energies and store them in the `state.hdf5` file. We just need to copy the input file and execute the calculation.

In [ ]:
!xtp_run -e einternal -f state.hdf5

## Compute site energy
In this step we will perform some *QMMM* calculations to compute the site energies. The `qmmm_mm.xml` file contains more options to perform the *MM* calculations.

In [ ]:
!xtp_parallel -e qmmm -o qmmm_mm.xml -f state.hdf5 -j "write"

The previous command generates a `qmmm_mm_jobs.xml` containing 3000 *MM* jobs to compute, if you examine that file, it should look something like:
```xml
<jobs>
        <job>
                <id>0</id>
                <tag>thiophene_0:n</tag>
                <input>
                        <site_energies>0:n</site_energies>
                        <regions>
                                <region>
                                        <id>0</id>
                                        <segments>0:n</segments>
                                </region>
                        </regions>
                </input>
                <status>AVAILABLE</status>
        </job>
```

Let us run just the first 4 jobs by settings all jobs `status` to `COMPLETE` except for the first four. This can be easily done with [sed](https://www.gnu.org/software/sed/manual/sed.html) as follows,

In [ ]:
!sed -i "s/AVAILABLE/COMPLETE/g" qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml

Now we can run the jobs and save the results in the state file

In [ ]:
!xtp_parallel -e qmmm -o qmmm_mm.xml -f state.hdf5 -j "run"
!xtp_parallel -e qmmm -o qmmm_mm.xml -f state.hdf5 -j "read"

## Site energy and pair energy analysis
In this step we generate an histogram and compute the correlation function of site energies and pair energy differences.

In [ ]:
!xtp_run -e eanalyze -f state.hdf5

You should now see a set of files prefixed with `eanalyze` containing the histrogram and correlation functions.

In [ ]:
!ls eanalyze*

## QM energy calculation
Our next task is to perform the qm calculations for each segment that we have stored in the hdf5 file. The calculations take place in 3 stages: write the jobs to a file, perform the computation and finally save the results to the state file. We created a small option file to make the calculation cheaper.

In [ ]:
!cat OPTIONFILES/eqm.xml

For the sake of computational time let just compute the `gw` approximation and the `singlet`. You can also request the `triplet` or `all`,  see the [gwbse sectionfor the eqm calculator](https://www.votca.org/xtp/eqm.html).

First we will write the job in a file and enable only the first 2 jobs

In [ ]:
!xtp_parallel -e eqm -o eqm.xml -f state.hdf5 -j "write"
!sed -i "s/AVAILABLE/COMPLETE/g" eqm.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' eqm.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' eqm.jobs

Now, let run these 2 jobs

Here we used some more options. `-o` allows us to read in a file with options. `-j` changes the writing to running in this case. `-x` determines how many cores should be used for each job. We can also run multiple jobs in parallel using `-p`

In [ ]:
!xtp_parallel -e eqm -o eqm.xml -f state.hdf5 -j run -x 4

## QM calculation for pairs
In the following step we will run QM calculations for each pair in the hdf5 file. As the calculations on the previous step, we will first write the jobs in a file, then run them and finally store the results in the state file.

As in the previous section, we set the GWBSE mode to `G0W0`and the `ranges` to `full`, but we compute only the `gw` approximation. We do not need the BSE results for the coupling calculations. For more information, check the [iqm calculator options](https://www.votca.org/xtp/iqm.html). We also want to compute the `singlet` couplings. 

Before running the calculations, we need to specify in the `iqm` input which states to read into the jobfile for each segment type.

Now, let's write the jobs to the file

In [ ]:
!xtp_parallel -e iqm -o iqm.xml -f state.hdf5 -s 0 -j "write"

From the jobs that we just write down, let's make available only the first job

In [ ]:
!sed -i "s/AVAILABLE/COMPLETE/g" iqm.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' iqm.jobs

Now we can run and store the jobs results

In [ ]:
!xtp_parallel -e iqm -o iqm.xml -f state.hdf5 -s 0 -j run -q 1 -x 4

Finally, we read the results into the state

In [ ]:
!xtp_parallel -e iqm -o iqm.xml -f state.hdf5 -j "read"

## QMMM Calculations
We will run the *QMMM* calculations we will use the pregenerated `qmmm.jobs` file in the current work directory, so we can directly run the calculations. We also provide an option file in the `OPTIONFILES` folder. In qmmm calculations you can use the `jobfile` tag inside the optionfile to modify options from the jobfile. Here we modify the size of the staticregion.

In [ ]:
!cat qmmm.xml

In the jobfile we then provide the specific option

In [ ]:
!cat qmmm.jobs

In [ ]:
!xtp_parallel -e qmmm -o qmmm.xml -f state.hdf5 -j run -x 4

We can if we want plot the spectra from both calculations, for which we have to read the energies and oscillator strengths from the checkpoint files. We need the h5py package for python for it.

In [ ]:
import h5py
import numpy as np

def getEnergies(orb):
    a=orb['region_0']['orbitals']['BSE_singlet']['eigenvalues'][()]
    a.flatten()
    return a.flatten()

def trans_sort(index):
    return int(index[3:])

def getOscillators(orb):
    energies=getEnergies(orb)
    transdip=[]
    for k in sorted(orb['region_0']['orbitals']['transition_dipoles'].keys(),key=trans_sort):
        transdip.append(np.array(orb['region_0']['orbitals']['transition_dipoles'][k][()]))
    d2=[]
    for b in transdip:
        d2.append(np.sum(b**2))    
    d2=np.array(d2)
    oscs=2/3.0*energies*d2
    return oscs

def getSpectrum(filename):
    orb=h5py.File(filename,'r')
    e=getEnergies(orb)*27.2114
    osc=getOscillators(orb)
    return e,osc

You will find the orb files in the `QMMM/frame_10000` folder.

In [ ]:
spectrum_static=getSpectrum("QMMM/frame_10000/job_1_static/checkpoint_iter_1.hdf5")
spectrum_vacuum=getSpectrum("QMMM/frame_10000/job_0_vacuum/checkpoint_iter_1.hdf5")

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]

In [ ]:
plt.vlines(spectrum_static[0],0,spectrum_static[1],label="static",color='r')
plt.vlines(spectrum_vacuum[0],0,spectrum_vacuum[1],label="vacuum",color='g')
plt.xlabel("energy [eV]")
plt.ylabel("intensity")
plt.legend()
plt.show()